In [2]:
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [3]:
df = pd.read_csv('dialise_fav_final3.csv', sep=';')
df.sample(5)

/tmp/ipykernel_10737/2972920874.py:1: DtypeWarning: Columns (20,37,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dialise_fav_final3.csv', sep=';')


,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,...,fav_ACF_FLEBIT,fav_ACF_HEMATO,fav_ACF_VEIAVI,fav_ACF_PULSO,fav_ACF_VEIDIA,fav_ACF_ARTDIA,fav_ACF_FREMIT,fav_AP_NATJUR,fav_repeticao,espera_fav
5805,201408,EP,430000,2226952,4314200153780,2014-08-01,305010107.0,2516.84,432120,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
281346,201808,PG,431710,2247488,4318210064070,2018-08-01,305010107.0,2602.14,431710,39,...,N,N,N,N,0000,0000,1.0,NaN,42.0,NaN
140177,201608,PG,431870,2232022,4316204728726,2016-08-01,305010107.0,2346.46,431870,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
561341,202206,EP,430000,2228602,4322207313017,2022-06-01,305010107.0,2858.13,430850,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
614993,202302,PG,430210,2241021,4323202798850,2023-02-01,305010107.0,2621.64,430210,5,...,N,N,N,N,0001,0001,1.0,3999.0,32.0,NaN


In [5]:
paleta = {
    'azul_principal': '#0077B6',
    'azul_claro'    : '#90E0EF',
    'ciano'         : '#48CAE4',
    'azul_escuro'   : '#023E8A',
    'cinza_texto'   : '#6C757D',
    'fundo'         : '#FFFFFF',
    'cinza_claro'   : '#F8F9FA'
}

# Pergunta 1

In [6]:
# Tempo de espera - Grafico de Linha + média por ano
df_espera = pd.DataFrame()
df_espera['data_primeira_dialise']      = df['ATD_DTPDR']
df_espera['cronico']                    = df['Cronico']
df_espera['data_atendimento_dialitico'] = df['AP_CMP']
df_espera['obito']                      = df['AP_OBITO']
df_espera['data_atendimento_fav']       = df['fav_AP_CMP']
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav
111124,2006-01-10,repetido,2016-03-01,0,NaN
195416,2016-06-01,repetido,2017-05-01,0,2016-07-01
387124,2015-05-21,repetido,2020-01-01,0,2015-08-01
140455,2014-09-26,repetido,2016-08-01,0,2014-11-01
527575,2021-07-16,repetido,2021-12-01,0,NaN
472051,2020-05-29,repetido,2021-03-01,0,2020-09-01
700659,2022-10-17,repetido,2024-03-01,0,2022-01-01
547538,2021-10-29,repetido,2022-02-01,0,2021-11-01
736430,2024-05-09,repetido,2024-09-01,0,2024-10-01
77490,2014-04-10,repetido,2015-09-01,0,NaN


In [7]:
# Para os casos de morte, podemos desconsiderar o tempo de espera (?)
df_espera_mortos_validos = df_espera[df_espera.obito == 1][df_espera.data_atendimento_fav.notna()]

# Remover todos os mortos
df_espera = df_espera[df_espera.obito != 1]

# Inserir apenas falecidos válidos
df_espera = pd.concat([df_espera, df_espera_mortos_validos])
df_espera.sample(10)

/tmp/ipykernel_10737/2896329900.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_espera_mortos_validos = df_espera[df_espera.obito == 1][df_espera.data_atendimento_fav.notna()]


,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav
400521,2016-06-11,repetido,2020-04-01,0,2016-06-01
551433,2020-09-01,repetido,2022-03-01,0,2021-01-01
345048,2019-05-18,agudo,2019-07-01,0,2019-06-01
473989,2021-02-25,agudo,2021-03-01,0,2021-04-01
55666,2012-01-14,repetido,2015-05-01,0,2021-03-01
186328,2015-08-24,repetido,2017-04-01,0,2015-10-01
163969,2016-09-12,cronico,2016-12-01,0,NaN
104751,2014-04-10,repetido,2016-02-01,0,NaN
727872,2023-03-29,repetido,2024-07-01,0,2024-03-01
490595,2011-03-10,repetido,2021-06-01,0,NaN


In [8]:
# Priorizar remover repetidos NaN
df_espera = df_espera[df_espera.cronico != 'repetido']
df_espera.sample(5)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav
429165,2020-07-23,agudo,2020-08-01,0,2020-08-01
326365,2019-02-27,agudo,2019-02-01,0,NaN
202269,2017-06-12,agudo,2017-06-01,0,2017-02-01
423325,2020-06-18,agudo,2020-07-01,0,NaN
289182,2018-06-09,cronico,2018-09-01,0,2018-08-01


In [9]:
# Remover observações que não são agudos e que não tiveram atendimento FAV
# Para garantir, vamos remover todos que não tiveram data_primeira_dialise também
df_espera = df_espera[df_espera.cronico != 'agudo']
df_espera = df_espera[df_espera.data_atendimento_fav.notna()]
df_espera = df_espera[df_espera.data_primeira_dialise.notna()]
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav
438612,2020-07-23,cronico,2020-10-01,0,2021-08-01
103576,2015-10-20,cronico,2016-01-01,0,2015-11-01
171170,2016-10-13,cronico,2017-01-01,0,2016-10-01
702689,2022-10-20,cronico,2024-04-01,0,2024-04-01
1972,2010-01-28,cronico,2014-08-01,0,2022-07-01
349414,2019-04-18,cronico,2019-07-01,0,2019-04-01
51328,2015-01-12,cronico,2015-04-01,0,2016-02-01
295991,2018-07-05,cronico,2018-10-01,0,2018-08-01
260037,2018-01-31,cronico,2018-04-01,0,2018-02-01
311042,2018-10-15,cronico,2019-01-01,0,2019-10-01


In [10]:
# Calcular tempo de espera
df_espera['data_atendimento_fav'] = pd.to_datetime(df_espera['data_atendimento_fav'])
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

# Calcular a diferença
df_espera['tempo_entre_fav_e_primeira_dialise'] = (
    df_espera['data_atendimento_fav'] - df_espera['data_primeira_dialise']
).dt.days  # em dias

df_espera['tempo_anos'] = df_espera['tempo_entre_fav_e_primeira_dialise'] / 365.25

# Remover esperas negativas
df_espera = df_espera[df_espera['tempo_entre_fav_e_primeira_dialise'] > 0]
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,tempo_entre_fav_e_primeira_dialise,tempo_anos
469849,2020-12-17,cronico,2021-03-01,0,2021-01-01,15,0.041068
698392,2023-12-28,cronico,2024-03-01,0,2024-01-01,4,0.010951
391946,2019-11-23,cronico,2020-02-01,0,2020-12-01,374,1.023956
389001,2019-10-14,cronico,2020-02-01,0,2020-05-01,200,0.547570
507145,2021-06-02,cronico,2021-09-01,0,2021-07-01,29,0.079398
5399,2013-04-18,cronico,2014-08-01,0,2014-11-01,562,1.538672
1488,2003-05-14,cronico,2014-08-01,0,2015-03-01,4309,11.797399
2223,2004-12-23,cronico,2014-08-01,0,2021-05-01,5973,16.353183
5798,2004-12-09,cronico,2014-08-01,0,2017-12-01,4740,12.977413
679195,2023-05-19,cronico,2023-12-01,0,2024-11-01,532,1.456537


In [11]:
# Manter dados apenas a partir de janeiro de 2014 devido criação do dataset

df_espera['data_atendimento_dialitico'] = pd.to_datetime(df_espera['data_atendimento_dialitico'])
df_espera['data_atendimento_fav'] = pd.to_datetime(df_espera['data_atendimento_fav'])
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

df_espera = df_espera[
        (df_espera['data_atendimento_dialitico'] >= '2014-01-01') &
        (df_espera['data_atendimento_fav'] >= '2014-01-01') &
        (df_espera['data_primeira_dialise'] >= '2014-01-01') ]

In [12]:
# Gráfico de tempo de espera
max_tempo = df_espera['tempo_entre_fav_e_primeira_dialise'].max()
bins = list(range(0, int(max_tempo) + 10, 10))

# Plotar histograma
fig1_p1 = px.histogram(
    df_espera,
    x='tempo_entre_fav_e_primeira_dialise',
    nbins=len(bins),
    title='Distribuição do Tempo de Espera para Atendimento no FAV',
    labels={
        'tempo_entre_fav_e_primeira_dialise': 'Tempo de espera (dias)',
        'count': 'Quantidade de pessoas'
    },
    color_discrete_sequence=[paleta['azul_principal']]  # Cor das barras
)

# Layout personalizado com a paleta
fig1_p1.update_layout(
    xaxis_title='Tempo de espera (dias)',
    yaxis_title='Quantidade de pessoas',
    bargap=0.05,
    template='plotly_white',
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    xaxis=dict(
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    ),
    yaxis=dict(
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    )
)

fig1_p1.show()

In [14]:
df_espera['ano_primeira_dialise'] = df_espera['data_primeira_dialise'].dt.year
df_agrupado = df_espera.groupby('ano_primeira_dialise')['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

fig2_p1 = px.bar(
    df_agrupado,
    x='ano_primeira_dialise',
    y='tempo_entre_fav_e_primeira_dialise',
    text='tempo_entre_fav_e_primeira_dialise',
    labels={
        'ano_primeira_dialise': 'Ano da Primeira Diálise',
        'tempo_entre_fav_e_primeira_dialise': 'Tempo médio de espera (dias)'
    },
    title='Tempo Médio de Espera por Ano da Primeira Diálise até a FAV',
    color='tempo_entre_fav_e_primeira_dialise',
    color_continuous_scale='Blues'
)

# Personalizar layout
fig2_p1.update_traces(texttemplate='%{text:.0f} dias', textposition='outside')

fig2_p1.update_layout(
    xaxis=dict(tickmode='linear'),
    yaxis_title='Tempo médio de espera (dias)',
    xaxis_title='Ano da primeira diálise',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor='white',
    width=900,
    height=500
)

fig2_p1.show()


# Pergunta 2

In [15]:
# Garantir que a coluna data_primeira_dialise está em datetime
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

# Criar coluna de ano-mês
df_espera['ano_mes'] = df_espera['data_primeira_dialise'].dt.to_period('M').astype(str)

# Agrupar por ano-mês
df_mes = df_espera.groupby('ano_mes')['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

# Converter ano-mês para datetime (para plotar no eixo)
df_mes['ano_mes_dt'] = pd.to_datetime(df_mes['ano_mes'])

# Gráfico de Linha (sem pontos)
fig1_p2 = go.Figure()

fig1_p2.add_trace(go.Scatter(
    x=df_mes['ano_mes_dt'],
    y=df_mes['tempo_entre_fav_e_primeira_dialise'],
    mode='lines',
    name='Tempo médio de espera',
    line=dict(color=paleta['azul_principal'], width=3)
))

# Tarja vermelha para COVID-19 (março de 2020 até dezembro de 2021)
fig1_p2.add_vrect(
    x0="2020-03-01", x1="2021-12-31",
    fillcolor="red", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="COVID-19", annotation_position="top left",
    annotation_font_size=12, annotation_font_color="red"
)

# Layout final com paleta aplicada
fig1_p2.update_layout(
    title='Média do Tempo de Espera por Mês',
    xaxis_title='Ano',
    yaxis_title='Tempo médio de espera (dias)',
    template='plotly_white',
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    width=1000,
    height=500,
    xaxis=dict(
        tickformat='%Y',
        dtick="M12",
        showgrid=True,
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    )
)

fig1_p2.show()

# 3) Existem perfis de pacientes que apresentam maior demora para a confecção da FAV?

# DASH Dashboard

In [74]:
# === App Dash ===
app = Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Dashboard - Tempo de Espera na FAV'),

    html.Div(children='''
        Análise dos tempos de espera entre a primeira diálise e o atendimento na FAV.
    '''),

    dcc.Graph(
        id='bar-plot',
        figure=fig1_p1
    ),
    dcc.Graph(
        id='histogram',
        figure=fig2_p1
    ),

    dcc.Graph(
    id='line-plot',
    figure=fig1_p2
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)